# Structured Output

In [1]:
from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)



True

### Method 1 : provide instruction in the prompt

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [3]:
"""
The usual "tell me a joke" LLM call.
"""

from src.ai_core.llm import get_llm
from src.ai_core.prompts import def_prompt


class Joke(BaseModel):
    the_joke: str = Field(description="a good joke")
    explanation: str = Field(description="explain why it's funny")
    rate: float = Field(description="rate how the joke is funny between 0 and 5")


parser = PydanticOutputParser(pydantic_object=Joke)

prompt_with_format = """
    tell me  a joke on {topic}
    ---
    {format_instructions}"""

structured_prompt = def_prompt(user=prompt_with_format).partial(
    format_instructions=parser.get_format_instructions(),
)

LLM_ID = None
structured_joke = structured_prompt | get_llm(llm_id=LLM_ID) | parser

r = structured_joke.invoke({"topic": "cat"})
print(r)

2025-04-02 10:47:02.610 | INFO     | src.utils.config_mngr:singleton:99 - selected config=training_edenai
2025-04-02 10:47:02.617 | INFO     | src.ai_core.cache:set_method:89 - LLM cache : InMemoryCache
2025-04-02 10:47:02.795 | INFO     | src.ai_core.llm:get_llm:497 - get LLM:'gpt_4omini_edenai'


Joke(
    the_joke='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!',
    explanation="It's funny because it plays on the double meaning of 'mouse'—the computer mouse and the animal 
mouse, which cats typically chase.",
    rate=4.0
)

In [4]:
print(structured_prompt)

ChatPromptTemplate(
    input_variables=['topic'],
    input_types={},
    partial_variables={
        'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema 
below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings",
"type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a 
well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not 
well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"the_joke": {"description": "a good joke", 
"title": "The Joke", "type": "string"}, "explanation": {"description": "explain why it\'s funny", "title": 
"Explanation", "type": "string"}, "rate": {"description": "rate how the joke is funny between 0 and 5", "title": 
"Rate", "type": "number"}}, "required": ["the_joke", "explanation", "rate"]}\n```'
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['format_instructions', 'topic'],
                input_types={},
                partial_variables={},
                template='tell me  a joke on {topic}\n---\n{format_instructions}'
            ),
            additional_kwargs={}
        )
    ]
)

In [5]:
# You can have a look at the generated prompt:
print(structured_prompt.invoke({"topic": "cat"}).messages[0].content)

tell me  a joke on cat
---
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": 
"array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": 
["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"the_joke": {"description": "a good joke", "title": "The Joke", "type": "string"}, "explanation": 
{"description": "explain why it's funny", "title": "Explanation", "type": "string"}, "rate": {"description": "rate 
how the joke is funny between 0 and 5", "title": "Rate", "type": "number"}}, "required": ["the_joke", 
"explanation", "rate"]}
```

### Method #2 : Use "with_structured_output"  (bases on function calls)

In [6]:
prompt = "tell me  a joke on {topic}"

# MODEL = None
MODEL = "gpt_4omini_edenai"  # better with Azure
chain = def_prompt(prompt) | get_llm(llm_id=MODEL).with_structured_output(Joke)
print(chain.invoke({"topic": "cat"}))

2025-04-02 10:51:44.695 | INFO     | src.ai_core.llm:get_llm:497 - get LLM:'gpt_4omini_edenai'


Joke(
    the_joke='Why was the cat sitting on the computer?',
    explanation='Because it wanted to keep an eye on the mouse!',
    rate=4.0
)

##  Assignement (Optional)
Rate the above joke.
Use https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/enum/ 


In [ ]:
from enum import Enum


class JokeRater(Enum):
    NOT_SO_GOOD = 0
    GOOD = 1
    VERY_GOOD = 2